# UFO Data Analysis Notebook:
#The Truth IS Out There!
## __Our two datasets are:  
1. **UFO_Sightings.csv**: A table with basic UFO sighting details like when and where they happened, and what shape the UFO was.  
2. **UFO_Truth.csv**: A table with more detailed UFO reports, including things like the weather and research outcomes.

**What You’ll Learn**  
- How to load data into Databricks.  
- How to look at data to spot problems (like missing or messy bits).  
- How to clean data so it’s usable.  
- How to join two tables together.  
- How to find patterns and show them with charts.  

# Setup
- You’ve uploaded the two CSV files (UFO_Sightings.csv and UFO_Truth.csv) to Databricks. To do this, go to the __"Catalog - DBFS - Upload"__. 
- click __"Catalog - Database Tables - Create Table,"__ and upload each file. Name them `ufo_sightings` and `ufo_truth`.  
- Your cluster (a computer in Databricks that runs your code) is running, look for a green "Running" status near your username.

# Tables
The tables we create are session only. So they will not be there after the session as they are __temp tables__. 
If you need to carry on with the notebook at a later date then you will have to create the tables all over again for a new session. 


# Loading Data into Databricks
**DataFrames**  
Data lives in files like CSV (comma-separated values), which are just tables saved as text. In Databricks, we load these into something called a **DataFrame**—A super-powered spreadsheet that can handle millions of rows. We’ll load our two UFO files and look at the first few rows to see what’s inside. 

**Why Do We Need To Do This?**  
Seeing the data helps us check if it loaded correctly, like weird symbols or empty rows.



In [0]:
#Load the UFO sightings table into a dataframe
ufo_sightings_df = spark.read.table("ufo_sightings_csv")
print("First 5 rows of UFO sightings:")
ufo_sightings_df.show(5)

#Load the UFO truth table into a dataframe
ufo_truth_df = spark.read.table("ufo_truth_csv")
print("First 5 rows of UFO truth:")
ufo_truth_df.show(5)

First 5 rows of UFO sightings:
+----------------+--------------------+-----+-------+--------+------------------+--------------------+--------------------+-----------+----------+------------+
|        datetime|                city|state|country|   shape|duration (seconds)|duration (hours/min)|            comments|date posted|  latitude|  longitude |
+----------------+--------------------+-----+-------+--------+------------------+--------------------+--------------------+-----------+----------+------------+
|10/10/1949 20:30|          san marcos|   tx|     us|cylinder|              2700|          45 minutes|This event took p...|  4/27/2004|29.8830556| -97.9411111|
|10/10/1949 21:00|        lackland afb|   tx|   null|   light|              7200|             1-2 hrs|1949 Lackland AFB...| 12/16/2005|  29.38421|  -98.581082|
|10/10/1955 17:00|chester (uk/england)| null|     gb|  circle|                20|          20 seconds|Green/Orange circ...|  1/21/2008|      53.2|   -2.916667|
|10/10/19

#Checking the Data’s Structure (Schema)
**Why?**  
Every DataFrame has a **schema**, which is like a blueprint showing the names of the columns (the headers in our table) and their **data types** (e.g., text, numbers, dates). We’ll print this to see what we’ve got and check if anything looks off—like numbers stored as text, which we can’t use for calculations yet.

### Understanding the schema tells us what cleaning we’ll need to do later, like changing text to numbers or dates.


In [0]:
#print the schema for both dataframes
print("UFO Sightings schema: ")
ufo_sightings_df.printSchema()

print("UFO Truth schema: ")
ufo_truth_df.printSchema()

UFO Sightings schema: 
root
 |-- datetime: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- shape: string (nullable = true)
 |-- duration (seconds): string (nullable = true)
 |-- duration (hours/min): string (nullable = true)
 |-- comments: string (nullable = true)
 |-- date posted: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude : string (nullable = true)

UFO Truth schema: 
root
 |-- reportedTimestamp: string (nullable = true)
 |-- eventDate: string (nullable = true)
 |-- eventTime: string (nullable = true)
 |-- shape: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- witnesses: string (nullable = true)
 |-- weather: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- sighting: string (nullable = true)
 

# Cleaning the Data Step-by-Step
**What’s Cleaning?**  
Real-world data is often messy—missing values, wrong formats, or typos. **Cleaning** means fixing these issues so we can trust our results. 

# Cleaning UFO_Sightings Data
**What We’re Fixing**  
- **Missing Values**: If key info like `country` or `duration (seconds)` is blank, that row isn’t very useful. We’ll remove it.  
- **Data Types**: `duration (seconds)` is text (e.g., "120"=String), but it should be a number (e.g., 120.0) so we can do maths with it, like finding averages.

**Why Do This?**  
Clean data means we can analyse it without errors or missing pieces messing things up.

In [0]:
from pyspark.sql.functions import col

#remove rows missing 'country' or 'durations (seconds)'
ufo_sightings_clean = ufo_sightings_df.dropna(subset=["country", "duration (seconds)"])

#change 'duration (seconds)' from text to a number (float)
ufo_sightings_clean = ufo_sightings_clean.withColumn("duration_seconds", col("duration (seconds)").cast("float"))

#check the updated schema
ufo_sightings_clean.printSchema()

root
 |-- datetime: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- shape: string (nullable = true)
 |-- duration (seconds): string (nullable = true)
 |-- duration (hours/min): string (nullable = true)
 |-- comments: string (nullable = true)
 |-- date posted: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude : string (nullable = true)
 |-- duration_seconds: float (nullable = true)



# Cleaning UFO_Truth Data
**What We’re Fixing**  
- **Missing Values**: We’ll drop rows missing `eventTime` because timing is important for understanding when UFOs appear.  
- **Data Types**: `eventTime` is __text/String__ (e.g., "14:30:00"), but we’ll turn it into a proper time and pull out the hour (e.g., 14) to check for night sightings.

**Why Do This?**  
Fixing `eventTime` lets us filter for specific times (like night) and ensures we’re not missing key details.


In [0]:
from pyspark.sql.functions import hour, to_timestamp

#remove rows missing 'eventTime'
ufo_truth_clean = ufo_truth_df.dropna(subset=['eventTime'])

#turn 'eventTime' into a time and extract the hour
ufo_truth_clean = ufo_truth_clean.withColumn("HourOccurred", hour(to_timestamp("eventTime", "HH:mm:ss")))

#peek at the cleaned data
ufo_truth_clean.show(5)

+--------------------+----------+---------+------+--------+---------+-------------+---------+----------+----------+------------+--------+----------------+-------+---------------+------------+
|   reportedTimestamp| eventDate|eventTime| shape|duration|witnesses|      weather|firstName|  lastName|  latitude|   longitude|sighting|physicalEvidence|contact|researchOutcome|HourOccurred|
+--------------------+----------+---------+------+--------+---------+-------------+---------+----------+----------+------------+--------+----------------+-------+---------------+------------+
|1977-04-04T04:02:...|1977-03-31|    23:46|circle|       4|        1|         rain|      Ila| Bashirian|47.3294444|-122.5788889|       Y|               N|      N|      explained|        null|
|1982-11-22T02:06:...|1982-11-15|    22:04|  disk|       4|        1|partly cloudy| Eriberto|Runolfsson| 52.664913|   -1.034894|       Y|               Y|      N|      explained|        null|
|1992-12-07T19:06:...|1992-12-07|    19:

# Joining the Two Tables
**Joining**  
We are merging two lists of UFO reports, datasets to get the full story. 

# The Truth Is Out There!
We’ll combine our __DataFrames__ using a shared column—here, `shape` (e.g., "circle" or "triangle"). This is called a **join**, and we’ll use an **inner join**, which keeps only rows where the `shape` matches in both tables. It’s like pairing up things.


In [0]:
# join the cleaned dataframes on 'shape'
joined_df = ufo_sightings_clean.join(ufo_truth_clean, ufo_sightings_clean.shape == ufo_truth_clean.shape, "inner")

# show the first 5 rows
joined_df.show(5)

+----------------+--------------------+-----+-------+------+------------------+--------------------+--------------------+-----------+--------+----------+----------------+--------------------+----------+---------+------+--------+---------+-------------+---------+-----------+----------+------------+--------+----------------+-------+---------------+------------+
|        datetime|                city|state|country| shape|duration (seconds)|duration (hours/min)|            comments|date posted|latitude|longitude |duration_seconds|   reportedTimestamp| eventDate|eventTime| shape|duration|witnesses|      weather|firstName|   lastName|  latitude|   longitude|sighting|physicalEvidence|contact|researchOutcome|HourOccurred|
+----------------+--------------------+-----+-------+------+------------------+--------------------+--------------------+-----------+--------+----------+----------------+--------------------+----------+---------+------+--------+---------+-------------+---------+-----------+--

# How Common Are UFO Shapes?
**We want the TRUTH!**  
We’ll count how many times each UFO shape appears in our joined data. Lets see how many types of UFO/UAP is out there!

**What are we doing here?**  
This shows us which shapes are most popular, giving us a starting point for understanding UFO trends.


In [0]:
from pyspark.sql.functions import count

#count each shape in the joined data
shapes_count = joined_df.groupBy(ufo_sightings_clean.shape.alias("Shape"))\
                        .agg(count("*").alias("count"))\
                        .orderBy("Count", ascending=False)

# Show the results
shapes_count.show()

+--------+--------+
|   Shape|   count|
+--------+--------+
|  circle|40617699|
|    disk|26503840|
|   light|24852972|
|triangle| 7474356|
|  sphere| 4826640|
|    oval|  653715|
| pyramid|     189|
+--------+--------+




# Nighttime Sightings 

**What We’re Doing**   

We’ll check which shapes are seen at night (after 8 PM) by filtering our data and counting them. It’s like asking, “Do some UFOs prefer the dark?”  

 

**Why This?**   

This could reveal patterns, like if "triangles" show up more at night.  

 

## Filter for Night Events: 

The first line selects only those UFO events that occurred at or after 8 PM (i.e. where HourOccurred is 20 or above). 

In [0]:

# filter for events at 8pm or laters
night_events = ufo_truth_clean.filter(ufo_truth_clean.HourOccurred >= 20)

# count shapes at night
night_events_count = night_events.groupBy("shape")\
                                 .agg(count("*").alias("NightCount"))\
                                 .orderBy("NightCount", ascending = False)

# show the results
night_events_count.show()

+-----+----------+
|shape|NightCount|
+-----+----------+
+-----+----------+




The first line filters the ufo_truth_clean DataFrame to retain only those rows where the event occurred at 8 PM or later (where HourOccurred is 20 or above). 

In [0]:
# check filtered events at or after 8pm
night_events = ufo_truth_clean.filter(ufo_truth_clean.HourOccurred >= 20)

night_events.show(5)

+-----------------+---------+---------+-----+--------+---------+-------+---------+--------+--------+---------+--------+----------------+-------+---------------+------------+
|reportedTimestamp|eventDate|eventTime|shape|duration|witnesses|weather|firstName|lastName|latitude|longitude|sighting|physicalEvidence|contact|researchOutcome|HourOccurred|
+-----------------+---------+---------+-----+--------+---------+-------+---------+--------+--------+---------+--------+----------------+-------+---------------+------------+
+-----------------+---------+---------+-----+--------+---------+-------+---------+--------+--------+---------+--------+----------------+-------+---------------+------------+




# Lets investigate a problem here, where we are not returning data! 

### Lets look at the time column and see what is going on. 

In [0]:
# inspect eventTime and HourOccurred columns
ufo_truth_clean.select("eventTime", "HourOccurred").show(5)

+---------+------------+
|eventTime|HourOccurred|
+---------+------------+
|    23:46|        null|
|    22:04|        null|
|    19:01|        null|
|    20:56|        null|
|    11:42|        null|
+---------+------------+
only showing top 5 rows




From Above the HourOccurred column is null. To extract the hour from eventTime failed. This happens when the format in the to_timestamp() function does not match your actual data. 

In [0]:
ufo_truth_clean.groupBy("HourOccurred").count().orderBy("HourOccurred").show()

+------------+-----+
|HourOccurred|count|
+------------+-----+
|        null|18000|
+------------+-----+




The parameter truncate=False ensures that the full content of the column is shown without shortening any text. It displays the complete time values (like "23:46") 

In [0]:
#show the full value here of the column
ufo_truth_clean.select("eventTime").show(5, truncate = False)

+---------+
|eventTime|
+---------+
|23:46    |
|22:04    |
|19:01    |
|20:56    |
|11:42    |
+---------+
only showing top 5 rows




# Which Cell was the culprit?? Which cell did we get wrong? 

 

## Fixing the time Stamp 

withColumn("HourOccurred", hour(to_timestamp("eventTime", "HH:mm"))) 

This takes the "eventTime" column, converts its string values (like "23:46") into timestamp values using the "HH:mm" format, then extracts the hour (e.g. 23) and creates a new column "HourOccurred" with these integer values. 


## (IT WAS CELL 10)

In [0]:
# adjust eventTime to a timestamp using the "HH:mm" format
from pyspark.sql.functions import hour, to_timestamp

ufo_truth_clean = ufo_truth_clean.withColumn("HourOccurred", hour(to_timestamp("eventTime", "HH:mm")))

#verify
ufo_truth_clean.select("eventTime", "HourOccurred").show(5, truncate = False)

+---------+------------+
|eventTime|HourOccurred|
+---------+------------+
|23:46    |23          |
|22:04    |22          |
|19:01    |19          |
|20:56    |20          |
|11:42    |11          |
+---------+------------+
only showing top 5 rows



In [0]:
# cell: analyse and count nighttime UFO events

# filter the data for events at or after 8pm
night_events = ufo_truth_clean.filter(ufo_truth_clean.HourOccurred >= 20)

# group by the 'shape' column and count the number of nighttime events 
night_events_count = night_events.groupBy("shape")\
                                 .agg(count("*").alias("NightCount"))\
                                 .orderBy("NightCount", ascending = False)

night_events_count.show()

display(night_events_count)

+--------+----------+
|   shape|NightCount|
+--------+----------+
|    disk|      1030|
|  circle|       977|
|   light|       293|
|  square|       291|
|  sphere|       173|
|triangle|       172|
|     box|        39|
|    oval|        33|
| pyramid|        25|
|    null|         1|
+--------+----------+



shape,NightCount
disk,1030
circle,977
light,293
square,291
sphere,173
triangle,172
box,39
oval,33
pyramid,25
null,1


Databricks visualization. Run in Databricks to view.


# Making Visualisations   

We will make __visualisations__ using the Databricks 'display()' fucntion 

 

- X-Axis: Choose the column that represents the category (e.g., shape). 

- Y-Axis: Choose the column that represents the numerical value (e.g., NightCount). 

- Other Options: You may have options like sorting or adding labels—configure these as desired. 

In [0]:
display(shapes_count)

display(night_events_count)

Shape,count
circle,40617699
disk,26503840
light,24852972
triangle,7474356
sphere,4826640
oval,653715
pyramid,189


Databricks visualization. Run in Databricks to view.

shape,NightCount
disk,1030
circle,977
light,293
square,291
sphere,173
triangle,172
box,39
oval,33
pyramid,25
null,1


Databricks visualization. Run in Databricks to view.


# Final Check of Clean Data 

**DataFrame**   

Let’s check our work by listing the columns in our cleaned `ufo_sightings_clean` DataFrame. Like a final inspection to make sure everything’s as expected. 

 

**What for?**   

It confirms we’ve added the right columns (like `duration_seconds`). 